<a href="https://colab.research.google.com/github/paul-kim88/pythonProject2/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install lxml[html_clean] readability-lxml

import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"#&iar=news&ia=news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, "html.parser")

        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})

        return results
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[\t\n]+', '\n', text)
    return text



if __name__ == "__main__":
    search_query = input("Type What you want : ")
    search_results = duckduckgo_scrape(search_query, '3d')

    results = []

    if search_results:
        print(f"Search Results for '{search_query}':")
        for result in search_results:
            if 'wiki' in result['link']:
              continue
            print(f"Title: {result['title']}")
            print(f"Link: {result['link']}")
            print("-" * 20)

            # Try to get the link, handling HTTP errors
            try:
                response = requests.get(result['link'])
                response.raise_for_status()  # Raise an exception for bad status codes

                soup = BeautifulSoup(response.content, "html.parser")
                doc = Document(response.content)
                print(doc.title())
                content = extract(doc.summary())
                results.append({"title": result['title'], "link": result['link'], "content": content})

            except requests.exceptions.HTTPError as e:
                print(f"HTTP Error fetching {result['link']}: {e}")
            except Exception as e:
                print(f"Error fetching or processing {result['link']}: {e}")
    search_df=pd.DataFrame(results)
    search_df

Type What you want : LG화학 친환경 소재
Search Results for 'LG화학 친환경 소재':
Title: LG화학 - LG Chem
Link: https://www.lgchem.com/main/index?lang=ko_KR
--------------------
LG화학
Title: 세계 최초 생분해성 신소재를 개발한 Lg화학! 친환경 소재를 개발해 자원 선순환에 앞장섭니다
Link: https://blog.lgchem.com/2020/11/16_biodegradable-material/
--------------------
세계 최초 생분해성 신소재를 개발한 LG화학! 친환경 소재를 개발해 자원 선순환에 앞장섭니다
Title: PDF Product Book - LG Chem
Link: https://www.lgchem.com/upload/file/sustainability/2023_LETZero_Productbook_ko.pdf
--------------------


In [ ]:
pd.DataFrame(results)


,title,link,content
0,[뉴스추적] 남은 헌재의 시간…한덕수·마은혁 변수 탄핵심판 선고 영향은? [MBN ...,https://www.youtube.com/watch?v=UH3KohoHGKY,AboutPressCopyrightContact usCreatorsAdvertise...
1,'한덕수 탄핵' 이르면 내일 결론…윤 선고 영향은?,https://news.sbs.co.kr/news/endPage.do?news_id...,\n \n &lt;앵커&gt; \n \n 최상목 권한대행이 마은혁 후보자 ...
2,"헌재, 한덕수 탄핵심판 90분 만에 변론 끝…증인 신청도 기각",https://news.sbs.co.kr/news/endPage.do?news_id...,\n \n ▲ 한덕수 국무총리 \n \n 헌법재판소가 한덕수 국무총리...
3,한덕수 총리 탄핵심판 첫 재판…'내란죄 수사기록' 두고 재판관과 충돌 : 네이트 뉴스,https://news.nate.com/view/20250113n31252,\n김형두 재판관 “헌재 규칙 근거해 송부촉탁 가능”‘내란 묵인·헌재 재판관 임명 ...
4,"尹대통령, 20일 헌재서 한덕수 총리 대면…형사재판도 시작 | 연합뉴스",https://www.yna.co.kr/view/AKR20250214135500004,\n홍장원·조지호도 증인신문…18일 9차 변론은 양쪽 의견진술尹측 기일변경 신청 변...
5,'한덕수 선고' 이번 주 가능성…윤 대통령 탄핵심판 변수,https://www.msn.com/ko-kr/%EC%A0%95%EC%B9%98/%...,\r\n \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...
